In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 2

In [7]:
# exact solution
def u_ex(x):  
    x_norm = torch.norm(x, dim = 1)
    u_x = torch.sin(pi/2*(1-x_norm)).reshape([x.size()[0],1])
    return u_x

In [8]:
def f(x):
    x_norm = torch.norm(x, dim = 1)
    u_temp = u_ex(x)
    f_temp = 0.25*pi**2*torch.sin(pi/2*(1-x_norm))+(dimension -1)*pi/(2*x_norm)*torch.cos(pi/2*(1-x_norm))  
    return f_temp.reshape([x.size()[0],1])+ u_temp**3

In [9]:
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.25*u_hat**4 - f_temp*u_hat)/x.size()[0]
    
    Nb = 200
    theta = 2*pi*(2*torch.rand(Nb)-1.0)
    y = torch.rand(Nb, dimension).cuda()
    y[:, 0] = torch.cos(theta)
    y[:, 1] = torch.sin(theta)
    part_2 = (torch.sum((model(y)-u_ex(y))**2)/y.size()[0])
    
    lambda1 = 50.0
    return part_1 + lambda1 * part_2 

In [10]:
Data_size = 2000
def Gendata():
    i=0
    notes = torch.tensor([])
    while notes.size()[0]< Data_size:
        temp_notes = 2*torch.rand(1000,dimension)-1
        index=torch.where(torch.norm(temp_notes,dim=1)<1)
        temp_notes = temp_notes[index]
        notes = torch.cat((notes,temp_notes),0)
    notes = notes[1:Data_size,:]
    return notes.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=2, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact)**2))/torch.sqrt(torch.sum((exact)**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_2D_Dirichlet_nonlinear_lambda50.0.npy", error_save)
np.save("DRM_relative_error_2D_Dirichlet_nonlinear_lambda50.0.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(5655.1155, device='cuda:0')
current relative error is:  tensor(12.7216, device='cuda:0')
current epoch is:  50
current loss is:  tensor(638.4959, device='cuda:0')
current relative error is:  tensor(4.9908, device='cuda:0')
current epoch is:  100
current loss is:  tensor(259.5155, device='cuda:0')
current relative error is:  tensor(2.8402, device='cuda:0')
current epoch is:  150
current loss is:  tensor(134.0295, device='cuda:0')
current relative error is:  tensor(1.8792, device='cuda:0')
current epoch is:  200
current loss is:  tensor(72.5709, device='cuda:0')
current relative error is:  tensor(1.2937, device='cuda:0')
current epoch is:  250
current loss is:  tensor(38.8766, device='cuda:0')
current relative error is:  tensor(0.9080, device='cuda:0')
current epoch is:  300
current loss is:  tensor(20.2150, device='cuda:0')
current relative error is:  tensor(0.7819, device='cuda:0')
current epoch is:  350
current loss is:  tensor(7.4938, dev

current epoch is:  3100
current loss is:  tensor(-0.4887, device='cuda:0')
current relative error is:  tensor(0.6637, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(-0.4851, device='cuda:0')
current relative error is:  tensor(0.6444, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(-0.4860, device='cuda:0')
current relative error is:  tensor(0.6301, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(-0.5199, device='cuda:0')
current relative error is:  tensor(0.6126, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(-0.5516, device='cuda:0')
current relative error is:  tensor(0.5950, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(-0.5647, device='cuda:0')
current relative error is:  tensor(0.5757, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(-0.5906, device='cuda:0')
current relative error is:  tensor(0.5591, device='cuda:0')
current epoch is:  3450
current loss is:  tensor(-0.637

current epoch is:  6150
current loss is:  tensor(-1.1145, device='cuda:0')
current relative error is:  tensor(0.0552, device='cuda:0')
current epoch is:  6200
current loss is:  tensor(-1.0615, device='cuda:0')
current relative error is:  tensor(0.0587, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(-1.0580, device='cuda:0')
current relative error is:  tensor(0.0555, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(-0.9870, device='cuda:0')
current relative error is:  tensor(0.0541, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(-1.0260, device='cuda:0')
current relative error is:  tensor(0.0601, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(-1.0444, device='cuda:0')
current relative error is:  tensor(0.0601, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(-0.9934, device='cuda:0')
current relative error is:  tensor(0.0575, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(-1.047

current epoch is:  9200
current loss is:  tensor(-1.0293, device='cuda:0')
current relative error is:  tensor(0.0463, device='cuda:0')
current epoch is:  9250
current loss is:  tensor(-1.0930, device='cuda:0')
current relative error is:  tensor(0.0444, device='cuda:0')
current epoch is:  9300
current loss is:  tensor(-1.0522, device='cuda:0')
current relative error is:  tensor(0.0502, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(-1.1032, device='cuda:0')
current relative error is:  tensor(0.0558, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(-0.9952, device='cuda:0')
current relative error is:  tensor(0.0556, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(-1.0657, device='cuda:0')
current relative error is:  tensor(0.0436, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(-1.0569, device='cuda:0')
current relative error is:  tensor(0.0472, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(-1.046

current epoch is:  12250
current loss is:  tensor(-1.0449, device='cuda:0')
current relative error is:  tensor(0.0542, device='cuda:0')
current epoch is:  12300
current loss is:  tensor(-1.1028, device='cuda:0')
current relative error is:  tensor(0.0580, device='cuda:0')
current epoch is:  12350
current loss is:  tensor(-1.0557, device='cuda:0')
current relative error is:  tensor(0.0534, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(-1.0253, device='cuda:0')
current relative error is:  tensor(0.0571, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(-1.0913, device='cuda:0')
current relative error is:  tensor(0.0473, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(-1.1011, device='cuda:0')
current relative error is:  tensor(0.0485, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(-0.9975, device='cuda:0')
current relative error is:  tensor(0.0614, device='cuda:0')
current epoch is:  12600
current loss is:  tenso

current epoch is:  15300
current loss is:  tensor(-1.0731, device='cuda:0')
current relative error is:  tensor(0.0412, device='cuda:0')
current epoch is:  15350
current loss is:  tensor(-1.0902, device='cuda:0')
current relative error is:  tensor(0.0609, device='cuda:0')
current epoch is:  15400
current loss is:  tensor(-1.0552, device='cuda:0')
current relative error is:  tensor(0.0471, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(-1.0372, device='cuda:0')
current relative error is:  tensor(0.0550, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(-1.0196, device='cuda:0')
current relative error is:  tensor(0.0588, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(-1.0826, device='cuda:0')
current relative error is:  tensor(0.0509, device='cuda:0')
current epoch is:  15600
current loss is:  tensor(-1.0403, device='cuda:0')
current relative error is:  tensor(0.0437, device='cuda:0')
current epoch is:  15650
current loss is:  tenso

current epoch is:  18350
current loss is:  tensor(-1.1156, device='cuda:0')
current relative error is:  tensor(0.0505, device='cuda:0')
current epoch is:  18400
current loss is:  tensor(-1.0637, device='cuda:0')
current relative error is:  tensor(0.0529, device='cuda:0')
current epoch is:  18450
current loss is:  tensor(-1.0401, device='cuda:0')
current relative error is:  tensor(0.0569, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(-1.0149, device='cuda:0')
current relative error is:  tensor(0.0512, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(-1.0155, device='cuda:0')
current relative error is:  tensor(0.0495, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(-1.0466, device='cuda:0')
current relative error is:  tensor(0.0483, device='cuda:0')
current epoch is:  18650
current loss is:  tensor(-1.0692, device='cuda:0')
current relative error is:  tensor(0.0491, device='cuda:0')
current epoch is:  18700
current loss is:  tenso